# Objet : Test opendata des données IRVE

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- l'outil fonctionne correctement (pas d'erreur identifiées, les temps de réponse restent néanmoins à améliorer).
- l'analyse identifie des données anormales qui seraient à corriger (ou à expliquer) -> cf exemple dans la dernière cellule
- le gain en taille de fichier varie de 50% (format texte non optimisé) à 80% (taille divisée par 5 !) dans un format optimisé et binaire. Ce niveau d'optimisation est tout à fait notable.
- l'analyse permet de (re)trouver la logique de dépendance entre les colonnes qui minimise les incohérences
- elle permet également de vérifier que les données respectent bien une structure imposée

## Usages possibles 
- les indicateurs utilisés permettent de qualifier le fichier csv et sont déployables simplement:
    - niveau de duplication des données
    - lien de dépendance entre les colonnes (permet un codage réduit des données).
- l'usage de format de données moins gourmand (sans dégradation des données) peut également être intéressant à déployer
- les schémas de données pourraient intégrer cette dépendance entre colonnes (ex. s'il y a une colonne "mois" et une colonne "trimestre", on peut indiquer que la colonne "trimestre" est "dérivée" de la colonne "mois", ou bien si on a une colonne "nom prénom" et une colonne "n° sécurité sociale", on peut indiquer que les deux colonnes sont "couplées". 
- la qualité des données peut faire l'objet d'un indicateur qui mesure l'écart (existant / attendu) des relations entre colonnes
- l'identification des enregistrements ne respectant pas une structure imposée permet de les éliminer ou de les corriger

## Autres points
- capacité ok de l'objet Iindexset à traiter des structures de données importantes (50 colonnes)
- indicateurs et fonctions de conversion pertinentes
- temps de réponse de la fonction 'coupling' à regarder
- tests complémentaires à effectuer sur d'autres jeux de données
- représentation graphique de la structure des données à regarder
- indicateur qualité à regarder

données utilisées : https://static.data.gouv.fr/resources/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/20220629-080611/consolidation-etalab-schema-irve-v-2.0.2-20220628.csv

------
## Initialisation
- lecture du fichier issu de l'api (quelques lignes incohérentes au niveau csv ont été éliminées au préalable)

In [1]:
from pprint import pprint
from collections import Counter
from time import time
import csv
import os
os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from util import util
from ilist import Ilist
from ESconstante import ES
from copy import copy

chemin = 'C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/validation/irve/'
file = chemin + 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv'

print('file size : ', os.stat(file).st_size)

file size :  7467241


In [2]:
t0 = time()
with open(file, newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    names = next(reader)
    data = []
    for row in reader: data.append(row)
data2 = util.list(list(zip(*data)))
print('data', time()-t0)

data 0.7302470207214355


----
## initialisation de l'objet Ilist
- l'initialisation pourrait être automatisée à partir du fichier csv
- identification de 64 775 valeurs différentes sur un total de 11 163 x 49 valeurs ("taux d'unicité" de 12%)
- la taille minimale serait de 1,4 Mo (données csv "quotées") pour un maximum de 9,6 Mo (données csv "quotées")

In [3]:
t0=time()
idxs = Ilist.Iext(data2, names)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs), len(idxs.idxlen), sum(idxs.idxlen), time()-t0)
t0=time()
fullsize = len(idxs.to_obj(encoded=True, fullcodec=True))
print('fullsize', fullsize, time()-t0)
t0=time()
minsize = len(idxs.to_obj(encoded=True, defaultcodec=True, fullcodec=True))
print('minsize', minsize, time()-t0)

idxs (len, lenlidx, sumcodec) :  11163 49 64775 6.532392978668213
fullsize 9682078 5.360515832901001
minsize 1390520 0.927593469619751


----
## format non optimisé
- le "taux d'unicité" reste à 12% (pas de modification des index)
- le "taux de codage" est de 30% (remplacement des données dupliquées par un entier)
- le gain de taille de fichier par rapport à un fichier "quoté" est de 61%
- l'analyse de la structure montre que les données sont principalement du type "linked" (non ou peu structuré)
- quelques colonnes sont de type "derived". Par exemple les index longitude(43) et latitude(44) sont bien dérivés de l'index coordonneesXY(13)
- le taux de couplage ("linkrate") pour chacun des index est très proche de 0, ce qui signifie que les données devraient être de type "derived" (lien de dépendance par exemple comme entre les trimestres et les mois) ou "coupled" (lien biunivoque).

In [4]:
t0=time()
defaultsize = len(idxs.to_obj(encoded=True, defaultcodec=True))
print('defaultsize', defaultsize, time()-t0)
print('indicator default : ', idxs.indicator(fullsize, defaultsize))
pprint(idxs.indexinfos(keys=['num', 'name', 'lencodec', 'parent', 'typecoupl'], base=True), width=120)
pprint(idxs.indexinfos(keys=['linkrate']))

defaultsize 3709049 1.0710499286651611
indicator default :  {'unique values': 64824, 'unicity level': 0.116, 'mean size': 5.239, 'object lightness': 0.302, 'gain': 0.617}
[{'lencodec': 481, 'name': 'nom_amenageur', 'num': 0, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 670, 'name': 'siren_amenageur', 'num': 1, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 388, 'name': 'contact_amenageur', 'num': 2, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 152, 'name': 'nom_operateur', 'num': 3, 'parent': 39, 'typecoupl': 'linked'},
 {'lencodec': 159, 'name': 'contact_operateur', 'num': 4, 'parent': 39, 'typecoupl': 'linked'},
 {'lencodec': 215, 'name': 'telephone_operateur', 'num': 5, 'parent': 13, 'typecoupl': 'linked'},
 {'lencodec': 764, 'name': 'nom_enseigne', 'num': 6, 'parent': 9, 'typecoupl': 'linked'},
 {'lencodec': 7060, 'name': 'id_station_itinerance', 'num': 7, 'parent': 15, 'typecoupl': 'linked'},
 {'lencodec': 3838, 'name': 'id_station_local', 'num': 8, 'parent': 16, '

----
## couplage
- transformation des index qui devraient être couplés en index couplés (tous les index sont bien maintenant de type "derived")


In [5]:
t0=time()
infos = idxs.coupling()
print('coupling', time()-t0)

coupling 38.52938199043274


----
## Format optimisé
- le "taux d'unicité" se dégrade légèrement (passage de 11,6% à 12,1%) par l'ajout d'index supplémentaires
- le "taux de codage" par contre passe de 30% à 16% de par l'optimisation 
- le gain de taille de fichier par rapport à un fichier "quoté" est maintenant de 74%
- l'utilisation d'un format binaire (codage CBOR pour Concise Binary Object Representation RFC 8949) permet d'améliorer encore le gain de taille de fichier (82%)

In [6]:
t0=time()
optimizesize = len(idxs.to_obj(indexinfos=infos, encoded=True))
print('optimizesize ', optimizesize, time()-t0)
print('indicator optimize : ', idxs.indicator(fullsize, optimizesize))
t0=time()
js = idxs.to_obj(indexinfos=infos, encoded=True, encode_format='cbor')
cborsize = len(js)
print('cborsize', cborsize, time()-t0)
print('indicator cbor : ', idxs.indicator(fullsize, cborsize))

optimizesize  2500273 7.705213785171509
indicator optimize :  {'unique values': 67357, 'unicity level': 0.121, 'mean size': 2.714, 'object lightness': 0.156, 'gain': 0.742}
cborsize 1720640 7.563085556030273
indicator cbor :  {'unique values': 67357, 'unicity level': 0.121, 'mean size': 1.125, 'object lightness': 0.065, 'gain': 0.822}


----
## Intégrité
- la transformation inverse des données binaires permet de vérifier qu'on retombe bien sur les mêmes données (pas de dégradation)

In [7]:
t0=time()
idxs2 = Ilist.from_obj(js)
print('fromcbor', len(idxs2), time()-t0)
t0=time()
verif = idxs2 == idxs
print('controle égalité :', verif, time()-t0)


fromcbor 11163 1.9655418395996094
controle égalité : True 5.853138446807861


----
## Exemple de données anormales
- l'index coordonneesXY(13) est lié à l'index nom_station(9) avec un taux de couplage très faible, par ailleurs, le nombre de valeurs de ces deux index sont très proches (4503 nom_station pour 4459 coordonneesXY), ce qui signifie que dans la majorité des cas, on associe de façon unique une station et une position
- les exemples ci-dessous montrent les écarts les plus important :
    - la position [1.106329, 49.474202] est associée à 10 stations
    - la station Camping Arinella est associée à 5 positions
    
- de même pour l'index coordonneesXY(13) qui est lié à l'index adresse_station(11), le taux de couplage est très faible avec 44 enregistrements sur 4456 en écart. On a par exemple quatre enregistrements avec la position (6.3491347, 47.3517596) et des adresses différentes ('58 Avenue du PrÃ©sident Kennedy 26', '58 Avenue du PrÃ©sident Kennedy 28', '58 Avenue du PrÃ©sident Kennedy 27', '58 Avenue du PrÃ©sident Kennedy 25')

In [8]:
champ = idxs.lindex
print('Couplage entre ', champ[13].name, ' et ', champ[9].name, ' : ', champ[13].couplinginfos(champ[9])['typecoupl'])
infosdefault = champ[9].couplinginfos(champ[13], default=True)
print('Ecart : ', infosdefault['disttomin'], 'positions sur ', infosdefault['distmin'], '\n') # moins de 1%
nom_station = champ[9].tostdcodec(full=False)
coordonneesXY = champ[13].tostdcodec(full=False) 
coordonneesXY.coupling(nom_station)
c = Counter(coordonneesXY.codec).most_common(5)
print('les 5 positions avec le plus de stations: ', c, '\n')
print('liste des stations associées à la position', c[0][0], ' : ', 
      set([nom_station[i] for i in coordonneesXY.recordfromvalue(c[0][0])]), '\n')
coordonneesXY = champ[13].tostdcodec(full=False)
nom_station.coupling(coordonneesXY)
c = Counter(nom_station.codec).most_common(5)
print('les 5 stations avec le plus de positions: ', c, '\n')
print('liste des positions associées à la station', c[0][0], ' : ', 
      set([coordonneesXY[i] for i in nom_station.recordfromvalue(c[0][0])]))

Couplage entre  coordonneesXY  et  nom_station  :  derived
Ecart :  44 positions sur  4503 

les 5 positions avec le plus de stations:  [('[1.106329, 49.474202]', 10), ('[3.080477, 50.675889]', 6), ('[1.313367, 49.137233]', 6), ('[2.523685, 48.9908]', 5), ('[3.1557445, 50.5161745]', 4)] 

liste des stations associées à la position [1.106329, 49.474202]  :  {'SCH09', 'SCH10', 'SCH01', 'SCH05', 'SCH07', 'SCH04', 'SCH06', 'SCH03', 'SCH08', 'SCH02'} 

les 5 stations avec le plus de positions:  [('Camping Arinella', 5), ('GUERET', 4), ('Noues de Sienne, Le Bourg', 3), ('Saint-Pierre de MaillÃ©', 2), ('ChÃ¢teau du MÃ©e - LE MEE SUR SEINE', 2)] 

liste des positions associées à la station Camping Arinella  :  {'[9.445073, 41.995246]', '[9.445072, 41.995246]', '[9.445071, 41.995246]', '[9.445075, 41.995246]', '[9.445074, 41.995246]'}


In [9]:
print('Couplage entre ', champ[11].name, ' et ', champ[13].name, ' : ', champ[11].couplinginfos(champ[13])['typecoupl'])
infosdefault = champ[13].couplinginfos(champ[11], default=True)
print('Ecart : ', infosdefault['disttomin'], 'positions sur ', infosdefault['distmin'], '\n') # moins de 1%
coordonneesXY = champ[13].tostdcodec(full=False)
adresse_station = champ[11].tostdcodec(full=False) 
adresse_station.coupling(coordonneesXY)
c = Counter(adresse_station.codec).most_common(5)
print('les 5 adresses avec le plus de positions : ', c, '\n')
print('liste des position associées à l adresse :', c[0][0], ' : ', 
      set([coordonneesXY[i] for i in adresse_station.recordfromvalue(c[0][0])]), '\n')
adresse_station = champ[11].tostdcodec(full=False)
coordonneesXY.coupling(adresse_station)
c = Counter(coordonneesXY.codec).most_common(5)
print('les 5 positions avec le plus d adresses: ', c, '\n')
print('liste des adresses associées à la position', c[0][0], ' : ', 
      set([adresse_station[i] for i in coordonneesXY.recordfromvalue(c[0][0])]))

Couplage entre  adresse_station  et  coordonneesXY  :  linked
Ecart :  41 positions sur  4459 

les 5 adresses avec le plus de positions :  [("rue de l'Ã©glise", 7), ('Place de la Mairie', 6), ('place de la mairie', 6), ('rue Grande', 5), ('grande rue', 5)] 

liste des position associées à l adresse : rue de l'Ã©glise  :  {'[2.9614005, 48.4189934]', '[2.5653149, 48.5803653]', '[2.7603113, 48.97101]', '[3.2265338, 48.6845989]', '[3.3512623, 48.4528516]', '[3.343567, 48.693841]', '[2.76525, 48.448495]'} 

les 5 positions avec le plus d adresses:  [('[6.3491347,47.3517596]', 4), ('[0.654826, 47.353958]', 4), ('', 3), ('[1.34918, 49.157966]', 3), ('[0.9605582,49.5355585]', 3)] 

liste des adresses associées à la position [6.3491347,47.3517596]  :  {'58 Avenue du PrÃ©sident Kennedy 28', '58 Avenue du PrÃ©sident Kennedy 27', '58 Avenue du PrÃ©sident Kennedy 25', '58 Avenue du PrÃ©sident Kennedy 26'}


----
## Détection des incohérences de relations entre champs
- la fonction coupling permet de réorganiser la structure des relations en isolant les enregistrements incohérents
- dans le cas ci-dessous, elle est appliquée de façon automatique (minimisation des incohérences)
- les incohérences minimales sont dans le cas présent de 12%

In [10]:
idxs3 = Ilist.from_obj(js)
idxs3.coupling()
duplic = idxs3.getduplicates(resindex=ES.filter)
#duplic = idxs3.getduplicates(idxs3.lname, ES.filter)
print('nombre d enregistrements incohérents : ', len(duplic))
idxs3.applyfilter()
print('nombre d enregistrements cohérents et liste des indexs non dérivés : ', len(idxs3), idxs3.primary)

nombre d enregistrements incohérents :  1310
nombre d enregistrements cohérents et liste des indexs non dérivés :  9853 [15]


----
## Vérification simple
- on peut vérifier par exemple qu'une position est associée à une unique station et que réciproquement chaque station n'a qu'une seule position (relation 1-1 entre les deux champs).

In [26]:
idxs4 = copy(idxs2)
champ = idxs4.lindex
notcoupl = champ[13].coupling(champ[9], derived=False)
print('nombre de pdc avec position/station non couplées : ', len(notcoupl))
print('\nliste des premières incohérences : ')
liste = []
for i in range(100): liste.append((champ[9][notcoupl[i]], champ[13][notcoupl[i]]))
pprint(set(liste), width=120)

nombre de pdc avec position/station non couplées :  154

liste des premières incohérences : 
{(' IntermarchÃ© - Gauville - 22kW AC ', '[1.759704, 49.780879]'),
 ('AEROVILLE - BALI MARKET', '[2.523685, 48.9908]'),
 ('Aventim Bouvines - Lot 1', '[3.199466, 50.574045]'),
 ('Aventim Bouvines - Lot 2', '[3.199466, 50.574045]'),
 ('IntermarchÃ© - Gauville - 22kW AC ', '[1.759704, 49.780879]'),
 ('LIDL BARENTIN', '[0.9605582,49.5355585]'),
 ('LIDL ST AUBIN SUR SCIE', '[0.9605582,49.5355585]'),
 ('LIDL YVETOT', '[0.9605582,49.5355585]'),
 ('M2023', '[2.298185, 43.212574]'),
 ('M2024', '[2.298185, 43.212574]'),
 ('M2025', '[2.298185, 43.212574]'),
 ('M2026', '[2.298185, 43.212574]'),
 ('M2042', '[1.095348, 49.475509]'),
 ('M2043', '[1.095348, 49.475509]'),
 ('M2045', '[1.095561, 49.474002]'),
 ('M2048', '[1.095561, 49.474002]'),
 ('M2050', '[1.313367, 49.137233]'),
 ('M2051', '[1.313367, 49.137233]'),
 ('M2052', '[1.313367, 49.137233]'),
 ('M2057', '[1.09061, 49.445346]'),
 ('M2058', '[1.09061,

## Vérification par rapport à une structure imposée
- une autre utilisation possible est de vérifier les données par rapport à un modèle de données défini
- dans cet exemple, on peut regrouper les colonnes suivant quatre entités (ceci revient à considérer les colonnes comme des attributs de chacune des entités) : les opérateurs (colonne 6), les aménageurs (colonne 1), les stations (colonne 9), les pdc (colonne 15) -> cf quatre premières lignes ci-dessous
- on peut également indiquer les dépendances entre les quatre entités (les opérateurs et aménageurs sont dérivés par rapport aux stations qui elles sont dérivées par rapport aux pdc.

In [14]:
row_operateur   = [6,3,4,5]
row_amenageur   = [1,0,2]
row_station     = [9,10,11,12,13,14,23,24,25,26,27,28,29,30,31,32,33,34,35,36,
                   43,44,45,46,47,48]
row_pdc         = [15,16,17,18,19,20,21,22,7,8,37,38,39,40,41,42]

champ = idxs2.lindex
operateur       = [champ[i] for i in row_operateur]
amenageur       = [champ[i] for i in row_amenageur]
station         = [champ[i] for i in row_station]
pdc             = [champ[i] for i in row_pdc]

operateur[0].coupling(operateur[1:])
amenageur[0].coupling(amenageur[1:])
station  [0].coupling(station  [1:])
pdc      [0].coupling(pdc      [1:])

station  [0].coupling([operateur[0], amenageur[0]])
pdc      [0].coupling(station[0])
pprint(idxs2.indexinfos(keys=['num', 'name', 'parent', 'typecoupl']), width=120)



[{'name': 'nom_amenageur', 'num': 0, 'parent': 1, 'typecoupl': 'derived'},
 {'name': 'siren_amenageur', 'num': 1, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'contact_amenageur', 'num': 2, 'parent': 1, 'typecoupl': 'derived'},
 {'name': 'nom_operateur', 'num': 3, 'parent': 6, 'typecoupl': 'derived'},
 {'name': 'contact_operateur', 'num': 4, 'parent': 6, 'typecoupl': 'derived'},
 {'name': 'telephone_operateur', 'num': 5, 'parent': 6, 'typecoupl': 'derived'},
 {'name': 'nom_enseigne', 'num': 6, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'id_station_itinerance', 'num': 7, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'id_station_local', 'num': 8, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'nom_station', 'num': 9, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'implantation_station', 'num': 10, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'adresse_station', 'num': 11, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'code_insee_commune', 'num': 12, 'parent': 9, 'typec

## Mise en cohérence des données
- l'application de la structure imposée permet d'identifie les enregistrements qui ne respectent pas la structure (cf exemples indiqués plus haut).
- dans l'exemple proposé, on identifie 48% des données ne respectant pas la structure
- on vérifie également qu'avec les données restantes, la structure est bien respectée

In [15]:
duplic = idxs2.getduplicates(idxs2.lname, ES.filter)
print(Counter(idxs2.lidx[49].keys))
idxs2.applyfilter()
idxs2.to_csv()
print(len(idxs2), idxs2.primary)
pprint(idxs2.indexinfos(keys=['num', 'name', 'parent', 'typecoupl']), width=120)

Counter({1: 5775, 0: 5388})
5775 [15]
[{'name': 'nom_amenageur', 'num': 0, 'parent': 1, 'typecoupl': 'derived'},
 {'name': 'siren_amenageur', 'num': 1, 'parent': 13, 'typecoupl': 'derived'},
 {'name': 'contact_amenageur', 'num': 2, 'parent': 1, 'typecoupl': 'derived'},
 {'name': 'nom_operateur', 'num': 3, 'parent': 39, 'typecoupl': 'derived'},
 {'name': 'contact_operateur', 'num': 4, 'parent': 39, 'typecoupl': 'derived'},
 {'name': 'telephone_operateur', 'num': 5, 'parent': 6, 'typecoupl': 'derived'},
 {'name': 'nom_enseigne', 'num': 6, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'id_station_itinerance', 'num': 7, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'id_station_local', 'num': 8, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'nom_station', 'num': 9, 'parent': 15, 'typecoupl': 'derived'},
 {'name': 'implantation_station', 'num': 10, 'parent': 43, 'typecoupl': 'derived'},
 {'name': 'adresse_station', 'num': 11, 'parent': 9, 'typecoupl': 'derived'},
 {'name': 'code_inse